In [4]:
%pylab inline
from scipy.special import j0, j1
import numpy as np
import itertools as itrt
import matplotlib.pyplot as plt
%config Completer.use_jedi = False

Populating the interactive namespace from numpy and matplotlib


In [5]:
def gamma_to_beta(gamma):
    return sqrt(1-1/gamma**2)
def kinetic_to_betagamma(energy, rest_energy):
    gamma = energy/rest_energy+1
    beta = gamma_to_beta(gamma)
    return beta, gamma

In [6]:
m = 1.007319468  #amu
c = 299.792458 # mm/ns
q = 1.178734E-5 ##1.6E-19 C => 4.8032047E-10 statC [cm^(3/2)*g^(1/2)*s^(-1)] => [mm^(3/2)*amu^(1/2)*ns^(-1)];
Px = np.random.normal(-3e-2, 3e-2, 200) #3e-2 amu*mm/ns corresponds to 93 keV
Py = np.random.uniform(-3e-2, 3e-2, 200)
Pz = np.random.uniform(3.4e2, 3.41e2, 200)  # approx 1 GeV +- 4 MeV, CAREFUL WE ARE IN LAB FRAME HERE
Pt = np.sqrt( Px**2+Py**2+Pz**2+m**2*c**2)
gamma = Pt/(m*c)
bx = Px/(gamma*m*c)
by = Py/(gamma*m*c)
bz = Pz/(gamma*m*c)
beta_avg  = np.sqrt(bx**2+by**2+bz**2)

x = numpy.random.normal(-50, 50, 200)
y = numpy.random.uniform(-50, 50, 200)
z = numpy.random.uniform(-50, 50, 200)
t = numpy.random.uniform(0, 100/(beta_avg*c), 200)

bdotx = bx*np.random.uniform(-1e-3,1e-3)  #just a guess, there is probably much more acceleration going on in the core
bdoty = by*np.random.uniform(-1e-3,1e-3) 
bdotz = bz*np.random.uniform(-1e-7,1e-7) #sloppy approximation to reflect much less deviation in logitudinal direction

In [7]:
init_beam = {'x':x, 'y':y, 'z':z, 't':t, 'Px':Px, 'Py':Py, 'Pz':Pz,'Pt':Pt,
             'bx':bx,'by':by,'bz':bz,'bdotx':bdotx,'bdoty':bdoty,'bdotz':bdotz,'gamma':gamma,'q':q} 

In [9]:
init_beam['x'][1]

-13.650502061398846

In [5]:
def dist_euclid(vector,vector_ext,index): 
    result = {}
    result['R'] = np.zeros_like(vector['x'])    
    result['nx'] = np.zeros_like(vector['x'])
    result['ny'] = np.zeros_like(vector['x'])    
    result['nz'] = np.zeros_like(vector['x'])    
    for j in range(len(vector_ext['x'])):
        result['R'][j] = np.sqrt( (vector['x'][index]-vector_ext['x'][j])**2+
                          (vector['y'][index]-vector_ext['y'][j])**2+
                          (vector['z'][index]-vector_ext['z'][j])**2 )
        result['nx'][j] = (vector['x'][index]-vector_ext['x'][j])/result['R'][j]
        result['ny'][j] = (vector['y'][index]-vector_ext['y'][j])/result['R'][j]
        result['nz'][j] = (vector['z'][index]-vector_ext['z'][j])/result['R'][j]
    return(result)

In [6]:
def step1(h, vector,vector_ext): # nhat includes R and fnhat components, need to generate this per particle pair
    for i in range(len(vector['x'])):   #iterating over all real particles OR all reflection points (these must be done in separate steps)            
            nhat = dist_euclid(vector,vector_ext,i)
        for j in range(len(vector_ext['x'])): #summing all external contributions (reflected particles and/or local particles)            
            result = {}
            result['x'] = vector['x']
            result['y'] = vector['y']
            result['z'] = vector['z']
            result['t'] = vector['t']
            result['Px'] = vector['Px'] + h/m*vector['q']*vector_ext['q']*vector['gamma']\
                        *1/(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz'])**2)\
                        *[ 1/nhat['R']**2*vector['bx']*np.dot(vector_ext['bx','by','bz'],vector('bx,by,bz'))\
                        -nhat['nx']*np.dot(vector_ext['bx','by','bz'],vector('bx,by,bz'))\
                        /(vector_ext['gamma']**2*nhat['R']**2*(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz'])))\
                        -nhat['nx']/(c*nhat['R'])*(np.dot(vector['bx','by','bz'],(vector_ext['bdotx','bdoty','bdotz']+vector_ext['gamma']**2\
                        *np.dot(vector_ext['bx','by','bz'],np.dot(vector_ext['bx','by','bz'],vector_ext['bdotx','bdoty','bdotz']) ) ) ) ) ]
            result['Py'] = vector['Py'] + h/m*vector['q']*vector_ext['q']*vector['gamma']\
                        *1/(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz'])**2)\
                        *[ 1/nhat['R']**2*vector['by']*np.dot(vector_ext['bx','by','bz'],vector('bx,by,bz'))\
                        -nhat['ny']*np.dot(vector_ext['bx','by','bz'],vector('bx,by,bz'))\
                        /(vector_ext['gamma']**2*nhat['R']**2*(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz'])))\
                        -nhat['ny']/(c*nhat['R'])*(np.dot(vector['bx','by','bz'],(vector_ext['bdotx','bdoty','bdotz']+vector_ext['gamma']**2\
                        *np.dot(vector_ext['bx','by','bz'],np.dot(vector_ext['bx','by','bz'],vector_ext['bdotx','bdoty','bdotz']) ) ) ) ) ]
            result['Pz'] = vector['Pz'] + h/m*vector['q']*vector_ext['q']*vector['gamma']\
                        *1/(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz'])**2)\
                        *[ 1/nhat['R']**2*vector['bz']*np.dot(vector_ext['bx','by','bz'],vector('bx,by,bz'))\
                        -nhat['nz']*np.dot(vector_ext['bx','by','bz'],vector('bx,by,bz'))\
                        /(vector_ext['gamma']**2*nhat['R']**2*(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz'])))\
                        -nhat['nz']/(c*nhat['R'])*(np.dot(vector['bx','by','bz'],(vector_ext['bdotx','bdoty','bdotz']+vector_ext['gamma']**2\
                        *np.dot(vector_ext['bx','by','bz'],np.dot(vector_ext['bx','by','bz'],vector_ext['bdotx','bdoty','bdotz']) ) ) ) ) ]
            result['Pt'] = vector['Pt'] + h/m*vector['q']*vector_ext['q']*vector['gamma'] \
                        *[((vector_ext['gamma']**2*np.dot(vector_ext['bx','by','bz'],vector_ext('bdotx,bdoty,bdotz')))/(cnhat['R']) \
                        - 1/nhat['R']**2) /(1 - np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz']))]
            result['gamma'] = result['P0']/(m*c**2)
            result['bx'] = result['Px']/(m*c*result['gamma'])
            result['by'] = result['Py']/(m*c*result['gamma'])
            result['bz'] = result['Pz']/(m*c*result['gamma'])
            result['bdotx'] = (result['Px']-vector['Px']) / h   #necessary history to treat as an external particle later (keep in lab frame?) 
            result['bdoty'] =(result['Py']-vector['Py']) / h
            result['bdotz'] = (result['Pz']-vector['Pz']) / h
            vector = result #will this work? need to update values here between each particle pair calculation
    return result
        

def step2(h, vector,vector_ext,nhat): 
        result = {}
        result['x'] = vector['x'] + h/m * (vector['Px']+vector['q']/c*vector_ext['q']*vector_ext['bx']\
                    / (vector_ext['gamma']*c*nhat['R']*(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz']))))
        result['y'] = vector['y'] + h/m * (vector['Py']+vector['q']/c*vector_ext['q']*vector_ext['by']\
                    / (vector_ext['gamma']*c*nhat['R']*(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz']))))
        result['z'] = vector['z'] + h/m * (vector['Pz']+vector['q']/c*vector_ext['q']*vector_ext['bz']\
                    / (vector_ext['gamma']*c*nhat['R']*(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz']))))
        result['t'] = vector['t'] + h/m * (vector['Pt']+vector['q']/c*vector_ext['q']\
                    / (vector_ext['gamma']*c*nhat['R']*(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz']))))
        result['Px'] = vector['Px']
        result['Py'] = vector['Py']
        result['Pz'] = vector['Pz']
        result['Pt'] = vector['Pt']
        return result

In [7]:
beam = init_beam
external_particle = ?????
distances = ?????
for i in range(1,100):
    beam = step1(h, init_beam,external_particle,distances)
    beam = step2(h, init_beam,external_particle,distances)

SyntaxError: invalid syntax (2119329478.py, line 2)

Needed is gaussian solid angle checker...

Example:

A ring-shaped strip of cavity wall near the aperture, with 50 test points going around in a circle. It is 15mm long and the test particle is
R= 2cm away. The ring of test points has $r_o$=35mm outer radius and $r_i$=25mm inner radius
<br> 
Solid angle = surface area/R$^2$ 
<br> 
or
<br> 
$\Omega_{ring}$ = A/R$^2$ = $\pi(r_o-r_i) / R^2{\,}$

Then for a gaussian sphere at $R$ surrounding the test particle, we have the full integrated charge:

$\Omega_{sphere} = 4\pi{\,}$sr

So the proportion of the full flux from the test particle is 

$\Omega_{ring} / \Omega_{sphere}$  = $(r_o-r_i) / (4 R^2){\,}$

And then dividing by the number of (evenly distributed) test points on the ring, $N=50$ we have

$q_{ring~point} = q_{total} *(r_o-r_i) / (4 R^2 N) = q_{total} / 800$

This result is then stored in ```vector_ext['q']```

##### Needed still:

A bootstrapper for initial integrator steps to estimate reflected potentials?
<br>
No! Just drift first -- with linear intra-bunch space charge if possible -- until reflected potentials arrive. But be careful to start with a comfortable margin before reflected energies become non-negligible.
<br>
<br>
A test for existence of external particles?
<br>
-yes, must default to drift mode for initial steps

800

##### Step 1 - Run the 'init_beam' as shown through some drift to generate a history, test on a FODO lattice if there is time to get a more realistic output distribution (maybe with some basic linear space charge). This includes generating a 'distances' vector sampling from particle histories at appropriate previous timesteps, for all particle pairs in the bunch  (plenty of pairwise distance calculators we can use out of the box). Some bootstrapping needed (or advanced potentials) before there are particle histories. 
  <br>

##### Step 2 - Generate a second 'init_beam' for test points along the aperture surface. These will have zero starting momentum. Save momentum values for minimization in an optimization routine.  (LOW COST - ONCE PER INTEGRATION STEP)

  <br>

##### Step 3 - Generate spline map (or some simpler gradient map) along the aperture surface to determine local angles of reflection (ONLY NEEDED ONCE PER SIMULATION, NOT PER INTEGRATION STEP)

  <br>

##### Step 4 - Generate a 'distances' vector, for all particles pairs between the bunch and the reflected bunch (which is built using the spline map and has opposite-sign charge). (VERY COSTLY - ONCE PER INTEGRATION STEP)

  <br>



##### Step 4 - Identify surface points experiencing strongest force and minimize. Maximize longitudinal component of reflected force. (ONCE PER SIMULATION AT FIRST, THEN MAYBE FINE TUNE PER INTEGRATION STEP)